In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from time import perf_counter
import torch
from torchinfo import summary
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from TumorDetection.data.loader import DataPathLoader
from TumorDetection.data.dataset import TorchDataset, Dataset
from TumorDetection.utils.dict_classes import DataPathDir, Device
from TumorDetection.models.efsnet import EFSNet
from TumorDetection.models.utils.trainer import Trainer

In [3]:
dp = DataPathLoader(DataPathDir.get('dir_path'))
paths = dp()
tr_paths, val_paths = train_test_split(paths, test_size=100, random_state=0, shuffle=True)
tr_td = TorchDataset(tr_paths)
val_td = TorchDataset(val_paths)

train_dataloader = DataLoader(tr_td,
                              32,
                              shuffle=True)
batch = next(iter(train_dataloader))
model = EFSNet()
summary(model, (1, 256, 256), batch_dim=0,
        col_names=("input_size", "output_size", "num_params", "params_percent"),
        depth=3,
        row_settings=["var_names"],
        device=Device.get('device'),
        verbose=1)

print(f'Device: {model.device}')
seg, lab = model(torch.rand((1, 1, 256, 256)).to(device=Device.get('device')))
images = 1000
it = perf_counter()
for _ in range(images):
    model(torch.rand((1, 1, 256, 256), device=Device.get('device')))
et = perf_counter()
print(f'Time per 1000 images: {et-it:.3f}')
print(f'FPS: {images/(et-it):.3f}')

True
True
Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Param %
EFSNet (EFSNet)                                              [1, 1, 256, 256]          [1, 2, 256, 256]          --                             --
├─Encoder (encoder)                                          [1, 1, 256, 256]          [1, 64, 64, 64]           --                             --
│    └─InitialBlock (initial_block)                          [1, 1, 256, 256]          [1, 16, 128, 128]         --                             --
│    │    └─ConvBlock (cb11)                                 [1, 1, 256, 256]          [1, 15, 128, 128]         166                         0.09%
│    │    └─Dropout2d (spr11)                                [1, 15, 128, 128]         [1, 15, 128, 128]         --                             --
│    │    └─MaxPool2d (mxp21)                                [1, 1, 256, 256]          [1, 1, 128, 128]     

In [ ]:
dp = DataPathLoader(DataPathDir.get('dir_path'))
paths = dp()
tr_paths, val_paths = train_test_split(paths, test_size=100, random_state=0, shuffle=True)
tr_td = TorchDataset(tr_paths)
val_td = TorchDataset(val_paths)

trainer = Trainer()
trainer(model=EFSNet(),
        train_data=tr_td,
        test_data=val_td)